In [ ]:
%load_ext jupyter_black

In [ ]:
import pandas as pd
import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pytz

from utils import plot_line, plot_quantiles

In [ ]:
DATA_DIR = "../trades/"
LOG_DIR = "../logs/"
PROCESSED_DIR = "./trades/"

BASE = "WIF"
QUOTE = "USDC"

BASE_PRICE_MULT = 10000
BASE_SIZE_MULT = 100

### Preprocessing

In [ ]:
data_path = f"{DATA_DIR}{BASE.lower()}-{QUOTE.lower()}.csv"
processed_data_path = f"{PROCESSED_DIR}{BASE.lower()}-{QUOTE.lower()}-analysis.csv"

df = pd.read_csv(data_path)

# df

In [ ]:
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
df["timestamp"] = df["timestamp"].dt.tz_localize("UTC").dt.tz_convert("US/Eastern")
df = df.sort_values(by="timestamp")
df["price"] = df["price"] / BASE_PRICE_MULT
df["size"] = df["size"] / BASE_SIZE_MULT


df.head(5)

In [ ]:
df.to_csv(processed_data_path, index=False)

### Overview

In [ ]:
def calculate_pnl(df):
    inventory = 0
    cash_flow = 0

    pnl_series = pd.Series(index=df["timestamp"])

    for index, row in df.iterrows():
        trade_size = row["size"]
        trade_value = trade_size * row["price"]

        if row["side"] == "buy":
            cash_flow -= trade_value
            inventory += trade_size
        else:
            cash_flow += trade_value
            inventory -= trade_size

        if inventory > 0:
            unrealized_pnl = cash_flow + (inventory * row["price"])
        else:
            unrealized_pnl = cash_flow + (inventory * row["price"])

        pnl_series.loc[row["timestamp"]] = unrealized_pnl

    pnl_series = pnl_series.dropna()
    return pnl_series

In [ ]:
df["inventory_change"] = df.apply(
    lambda row: row["size"] if row["side"] == "buy" else -row["size"], axis=1
)
df["cumulative_inventory"] = df["inventory_change"].cumsum()
pnl_series = calculate_pnl(df)
flat_pnl_series = pd.Series(0, index=pnl_series.index)
inventory_series = df.set_index("timestamp")["cumulative_inventory"]
flat_inventory_series = pd.Series(0, index=inventory_series.index)
time_diffs = df["timestamp"].diff().dt.total_seconds()
time_diffs = time_diffs.dropna()

In [ ]:
duration = df["timestamp"].max() - df["timestamp"].min()
hours = duration.components.hours
minutes = duration.components.minutes

trade_volume = df["size"] * df["price"]
quote_volume = trade_volume.sum()
base_volume = df["size"].sum()

In [ ]:
print(f"Made {len(df)} trades over {hours}h {minutes}m")
print(f"Total volume ({QUOTE}): {quote_volume:.2f}")
print(f"Total volume ({BASE}): {base_volume}")
plot_line(
    df["timestamp"], [pnl_series, flat_pnl_series], ["PnL", "Break Even"], "Time", "PnL"
)
plot_line(
    df["timestamp"],
    [inventory_series, flat_inventory_series],
    ["Inventory", "Flat"],
    "Time",
    "Inventory",
)
plot_quantiles(time_diffs, bins=50, name="Time between Trades (s)")
plot_quantiles(df["size"], bins=50, name="Size")

In [ ]:
df.tail(5)